Last Man Standing - Optimal Strategies

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1234)
import pandas as pd
import itertools

from matplotlib.pyplot import figure, show
import pprint as pp

### Implementing Gameweeks

In [60]:
df = pd.read_csv("23_24_epl.csv")
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,NaN,NaN,1.92,1.95
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88


In [61]:
# Ensure the 'Date' column is parsed correctly
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Sort by Date
df = df.sort_values(by='Date').reset_index(drop=True)

# Initialize variables
gameweek = 1
gameweek_column = [None] * len(df)  # Placeholder for gameweek assignments
unassigned_matches = set(range(len(df)))  # Track matches not yet assigned to any gameweek

# Process gameweeks
while unassigned_matches:
    teams_in_gameweek = set()  # Track teams in the current gameweek
    matches_in_gameweek = []  # Track match indices in the current gameweek

    # Iterate through unassigned matches
    for match_index in sorted(unassigned_matches):
        home_team = df.loc[match_index, 'HomeTeam']
        away_team = df.loc[match_index, 'AwayTeam']

        # Check if both teams are available in the current gameweek
        if home_team not in teams_in_gameweek and away_team not in teams_in_gameweek:
            # Assign match to the current gameweek
            gameweek_column[match_index] = gameweek
            teams_in_gameweek.update([home_team, away_team])
            matches_in_gameweek.append(match_index)

            # Stop adding matches if we've reached 10 for this gameweek
            if len(matches_in_gameweek) == 10:
                break

    # Remove assigned matches from the unassigned set
    unassigned_matches -= set(matches_in_gameweek)

    # Increment gameweek for the next iteration
    gameweek += 1

# Add the gameweek column to the DataFrame
df['Gameweek'] = gameweek_column

# Save the updated DataFrame
df.to_csv("epl_games_with_gameweeks.csv", index=False)

# Display the counts for each gameweek
print(df['Gameweek'].value_counts().sort_index())


Gameweek
1     10
2     10
3     10
4     10
5     10
6     10
7     10
8     10
9     10
10    10
11    10
12    10
13    10
14    10
15    10
16    10
17    10
18    10
19    10
20    10
21    10
22    10
23    10
24    10
25    10
26    10
27    10
28    10
29     9
30     9
31     9
32     9
33    10
34     9
35     9
36     8
37     8
38     9
39     8
40     3
Name: count, dtype: int64


In [62]:
# Group matches by gameweek and check for unique teams
for gameweek, group in df.groupby('Gameweek'):
    # Combine HomeTeam and AwayTeam columns to get all teams in the gameweek
    teams = pd.concat([group['HomeTeam'], group['AwayTeam']])
    unique_teams = teams.nunique()  # Count unique teams

    # Print the result for each gameweek
    if unique_teams == 20:
        print(f"Gameweek {gameweek}: {unique_teams} unique teams ✅")
    else:
        print(f"Gameweek {gameweek}: {unique_teams} unique teams ❌ (Check this gameweek)")


Gameweek 1: 20 unique teams ✅
Gameweek 2: 20 unique teams ✅
Gameweek 3: 20 unique teams ✅
Gameweek 4: 20 unique teams ✅
Gameweek 5: 20 unique teams ✅
Gameweek 6: 20 unique teams ✅
Gameweek 7: 20 unique teams ✅
Gameweek 8: 20 unique teams ✅
Gameweek 9: 20 unique teams ✅
Gameweek 10: 20 unique teams ✅
Gameweek 11: 20 unique teams ✅
Gameweek 12: 20 unique teams ✅
Gameweek 13: 20 unique teams ✅
Gameweek 14: 20 unique teams ✅
Gameweek 15: 20 unique teams ✅
Gameweek 16: 20 unique teams ✅
Gameweek 17: 20 unique teams ✅
Gameweek 18: 20 unique teams ✅
Gameweek 19: 20 unique teams ✅
Gameweek 20: 20 unique teams ✅
Gameweek 21: 20 unique teams ✅
Gameweek 22: 20 unique teams ✅
Gameweek 23: 20 unique teams ✅
Gameweek 24: 20 unique teams ✅
Gameweek 25: 20 unique teams ✅
Gameweek 26: 20 unique teams ✅
Gameweek 27: 20 unique teams ✅
Gameweek 28: 20 unique teams ✅
Gameweek 29: 18 unique teams ❌ (Check this gameweek)
Gameweek 30: 18 unique teams ❌ (Check this gameweek)
Gameweek 31: 18 unique teams ❌ (Che

Let's just use the first 19 gameweeks to get the data for the first 19 weeks of the season.

In [63]:
wanted_columns = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "B365H", "B365D", "B365A", "Gameweek"]

football_data = df[wanted_columns][df['Gameweek'] <= 19]

football_data.head(30)

#length of dataframe
print(len(football_data))



190


Great, so we now have a dataframe with the first 19 gameweeks of the season, with the home and away teams, the number of goals scored by each team, and the bookmaker odds for the match. WLOG, we can consider a solution to this problem as a solution to our entire problem. What do we want to do next?

1. Convert the bookmakers odds into probabilities
2. Give some results of simulations of Last Man Standing. 

Simulation. 
At the start of each gameweek, we simulate our choice and the choices of the other M-1 players. We assume that other players follow a greedy sampling strategy - otherwise they would all pick the same player each time. We then benchmark against the actual results that we can see. A singular simulation will produce one winner. We should do loads - say 100, and then we may be able to see if our strategy is actually pretty good. 

We will perform a dynamic programming solution to this problem. At each gameweek, we will:
- Simulate 5 game weeks ahead (assuming all players according to the greedy sampling strategy)
- Calculate the maximal path through our graph of probabilities
- Take the first pick that aligns with this maximal 
- Run this simulation again at each gameweek. 

Converting the bookmakers odds into probabilities.

In [ ]:
# for all B365H, B365D, B365A, we want to convert to probabilities. 
# we can do this by taking the value, and dividing by the sum of the values
# we can do this for each row in the dataframe. 

# create a new dataframe with the probabilities
football_data_with_probabilities = football_data.copy()

# for each row in the dataframe, we want to convert the bookmakers odds to probabilities
for index, row in football_data_with_probabilities.iterrows():
    football_data_with_probabilities.loc[index, 'B365H'] = row['B365H'] / (row['B365H'] + row['B365D'] + row['B365A'])
    football_data_with_probabilities.loc[index, 'B365D'] = row['B365D'] / (row['B365H'] + row['B365D'] + row['B365A'])
    football_data_with_probabilities.loc[index, 'B365A'] = row['B365A'] / (row['B365H'] + row['B365D'] + row['B365A'])

print(football_data_with_probabilities)

#now lets rename the columns to be more descriptive
football_data_with_probabilities.rename(columns={'B365H': 'HomeWinProbability', 'B365D': 'DrawProbability', 'B365A': 'AwayWinProbability'}, inplace=True)

print(football_data_with_probabilities)

        HomeTeam       AwayTeam  FTHG  FTAG     B365H     B365D     B365A  \
0        Burnley       Man City     0     3  0.539447  0.370870  0.089683   
1        Arsenal  Nott'm Forest     2     1  0.050906  0.301984  0.647110   
2    Bournemouth       West Ham     1     1  0.312139  0.393064  0.294798   
3       Brighton          Luton     4     1  0.084018  0.347442  0.568541   
4        Everton         Fulham     0     1  0.247191  0.382022  0.370787   
..           ...            ...   ...   ...       ...       ...       ...   
185      Everton       Man City     1     3  0.507951  0.353357  0.138693   
186     Brighton      Tottenham     4     2  0.287356  0.413793  0.298851   
187      Arsenal       West Ham     0     2  0.092812  0.383810  0.523378   
227    Brentford       Man City     1     3  0.532319  0.361217  0.106464   
278  Bournemouth          Luton     4     3  0.125000  0.395833  0.479167   

     Gameweek  
0           1  
1           1  
2           1  
3          

In [ ]:
# Let's create a simulation for multiple agents
num_agents = 10
num_gameweeks = 19

# Create a dictionary to track which teams each agent has picked
agent_picks = {i: set() for i in range(num_agents)}

# Function to simulate one gameweek for all agents
def simulate_gameweek(gameweek_data, agent_picks):
    
    results = {}
    
    for agent_id in range(num_agents):
        # Get available matches (teams not yet picked by this agent)
        available_matches = gameweek_data[
            ~((gameweek_data['HomeTeam'].isin(agent_picks[agent_id])) | 
              (gameweek_data['AwayTeam'].isin(agent_picks[agent_id])))
        ]
        
        if len(available_matches) == 0:
            results[agent_id] = 'eliminated'
            continue

        # Get available home teams and their win probabilities
        available_teams = available_matches['HomeTeam'].tolist()
        probabilities = available_matches['HomeWinProbability'].tolist()

        # Softmax the probabilities
        probabilities = np.exp(probabilities) / np.sum(np.exp(probabilities))   

        # Randomly pick a team according to the probability distribution
        chosen_team = np.random.choice(available_teams, p=probabilities)
        
        # Record the pick
        agent_picks[agent_id].add(chosen_team)

        # Get the match result for the chosen team
        match_row = available_matches[available_matches['HomeTeam'] == chosen_team].iloc[0]
        actual_result = match_row['FTHG'] > match_row['FTAG']
        
        # Record if agent survived this week
        results[agent_id] = 'survived' if actual_result.item() else 'eliminated'
    
    return results

# Run simulation through all gameweeks
surviving_agents = set(range(num_agents))

for week in range(1, num_gameweeks + 1):
    gameweek_data = football_data_with_probabilities[football_data_with_probabilities['Gameweek'] == week]
    results = simulate_gameweek(gameweek_data, agent_picks)
    
    # Update surviving agents
    surviving_agents = {agent for agent in surviving_agents 
                       if results.get(agent) == 'survived'}
    
    print(f"After week {week}: {len(surviving_agents)} agents remaining")

print(f"\nFinal survivors: {len(surviving_agents)}")

After week 1: 2 agents remaining
After week 2: 1 agents remaining
After week 3: 0 agents remaining
After week 4: 0 agents remaining
After week 5: 0 agents remaining
After week 6: 0 agents remaining
After week 7: 0 agents remaining
After week 8: 0 agents remaining
After week 9: 0 agents remaining
After week 10: 0 agents remaining
After week 11: 0 agents remaining
After week 12: 0 agents remaining
After week 13: 0 agents remaining
After week 14: 0 agents remaining
After week 15: 0 agents remaining
After week 16: 0 agents remaining
After week 17: 0 agents remaining
After week 18: 0 agents remaining
After week 19: 0 agents remaining

Final survivors: 0


In [78]:
# now lets inspect the policy of the agents
# we can do this by looking at the picks of the agents
# the code for that is below:

print(agent_picks)

{0: {"Nott'm Forest", 'Burnley', 'Bournemouth', 'Fulham', 'Arsenal', 'Chelsea', 'Wolves', 'Man City', 'Everton', 'Sheffield United', 'Luton', 'Brighton', 'Aston Villa', 'West Ham', 'Liverpool', 'Newcastle'}, 1: {"Nott'm Forest", 'Bournemouth', 'Fulham', 'Burnley', 'Tottenham', 'Arsenal', 'Chelsea', 'Everton', 'Crystal Palace', 'Sheffield United', 'Luton', 'Brentford', 'Brighton', 'Aston Villa', 'West Ham', 'Liverpool'}, 2: {"Nott'm Forest", 'Bournemouth', 'Burnley', 'Tottenham', 'Arsenal', 'Chelsea', 'Man City', 'Crystal Palace', 'Sheffield United', 'Luton', 'Brentford', 'Brighton', 'West Ham', 'Newcastle', 'Man United'}, 3: {"Nott'm Forest", 'Bournemouth', 'Burnley', 'Tottenham', 'Arsenal', 'Chelsea', 'Man City', 'Everton', 'Crystal Palace', 'Sheffield United', 'Luton', 'Brentford', 'Aston Villa', 'West Ham', 'Liverpool', 'Newcastle', 'Man United'}, 4: {'Bournemouth', 'Burnley', 'Fulham', 'Tottenham', 'Wolves', 'Chelsea', 'Man City', 'Everton', 'Sheffield United', 'Luton', 'Brighton',

We want to simulate the actions of 1000 agents, and see if we can find a strategy that is better than the greedy sampling strategy. 